<a href="https://colab.research.google.com/github/pedrotpa/AulaPraticaTDD/blob/master/ncn_pytorch_cora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/tkipf/pygcn/master/data/cora/cora.cites -P data/cora
!wget https://raw.githubusercontent.com/tkipf/pygcn/master/data/cora/cora.content -P data/cora

--2020-03-22 17:27:04--  https://raw.githubusercontent.com/tkipf/pygcn/master/data/cora/cora.cites
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69928 (68K) [text/plain]
Saving to: ‘data/cora/cora.cites’

cora.cites          100%[===================>]  68.29K  --.-KB/s    in 0.02s   

2020-03-22 17:27:04 (3.00 MB/s) - ‘data/cora/cora.cites’ saved [69928/69928]

--2020-03-22 17:27:07--  https://raw.githubusercontent.com/tkipf/pygcn/master/data/cora/cora.content
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7823427 (7.5M) [text/plain]
Sav

In [0]:
import numpy as np
import pandas as pd

def load_cora(path="data/cora/"):
    path="data/cora/"
    content = pd.read_csv(path+"cora.content", sep='\t', header=None, index_col=False)
    node_idx = content[0].to_numpy().astype(np.int32)
    idx_map = {j:i for i,j in enumerate(node_idx)}

    del node_idx

    node_features= content.iloc[:,1:-1].to_numpy().astype(np.int8)

    label_column = content.columns[-1]
    # content[label_column] = pd.Categorical(content[label_column])
    # node_labels = pd.get_dummies(content[label_column]).to_numpy().astype(np.int8)
    node_labels = pd.Categorical(content[label_column]).codes
    del content

    cites = np.genfromtxt(path+"cora.cites",delimiter='\t', dtype=np.int32)
    edges = np.array(list(map(idx_map.get, cites.flatten())),
                        dtype=np.int32).reshape(cites.shape)

    adj = np.zeros((len(idx_map),len(idx_map), 2), dtype=np.float32)
    adj[edges[:,0], edges[:,1],0]=1
    adj[edges[:,1], edges[:,0],1]=1

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    return adj, node_features, node_labels, idx_train, idx_val, idx_test

In [0]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [0]:
!nvidia-smi

Fri Mar 20 23:32:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NodeConv(nn.Module):
    def __init__(self,in_size, out_size, edges_size = 1,bias=True, device='cpu'):
        super(NodeConv, self).__init__()
        self.in_size = in_size;
        self.out_size = out_size
        self.edges_size = edges_size
        self.kernel = nn.parameter.Parameter(torch.FloatTensor(out_size, edges_size*in_size).to(device))
        var = 2./(self.kernel.size(1)+self.kernel.size(0))
        self.kernel.data.normal_(0,var)
        if bias:
            self.bias=nn.parameter.Parameter(torch.FloatTensor(out_size).to(device))
            self.bias.data.normal_(0,var)
        else:
            self.register_parameter("bias", None)

    def forward(self, X, adj):
        X = X.T@adj
        X = X.view(X.shape[0], -1)
        X = X@self.kernel.T
        if self.bias is not None:
            return X + self.bias
        else:
            return X

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_size) + ' -> ' \
               + str(self.out_size) + ')'

class NCN(nn.Module):
    def __init__(self, in_size, n_hid, out_size, edges_size, dropout, bias=True, device='cpu'):
        super(NCN, self).__init__()
        self.ncn1 = NodeConv(in_size, n_hid, edges_size=edges_size, bias=bias, device=device)
        self.ncn2 = NodeConv(n_hid, out_size, edges_size=edges_size, bias=bias, device=device)
        self.dropout = dropout

    def forward(self, X, adj):
        X = F.relu(self.ncn1(X,adj))
        X = F.dropout(X, self.dropout, training=self.training)
        return F.log_softmax(self.ncn2(X, adj), dim=1)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NodeConv(nn.Module):
    def __init__(self,in_size, out_size, edges_size = 1,bias=True, device='cpu'):
        super(NodeConv, self).__init__()
        self.in_size = in_size;
        self.out_size = out_size
        self.edges_size = edges_size
        self.kernel = nn.parameter.Parameter(torch.FloatTensor(out_size, edges_size*in_size).to(device))
        var = 2./(self.kernel.size(1)+self.kernel.size(0))
        self.kernel.data.normal_(0,var)
        if bias:
            self.bias=nn.parameter.Parameter(torch.FloatTensor(out_size).to(device))
            self.bias.data.normal_(0,var)
        else:
            self.register_parameter("bias", None)

    def forward(self, X, adj):
        X = X.T@adj
        X = X.view(X.shape[0], -1)
        X = X@self.kernel.T
        if self.bias is not None:
            return X + self.bias
        else:
            return X

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_size) + ' -> ' \
               + str(self.out_size) + ')'

class NCN(nn.Module):
    def __init__(self, in_size, n_hid, out_size, edges_size, dropout, bias=True, device='cpu'):
        super(NCN, self).__init__()
        self.ncn1 = NodeConv(in_size, n_hid, edges_size=edges_size, bias=bias, device=device)
        self.ncn2 = NodeConv(n_hid, out_size, edges_size=edges_size, bias=bias, device=device)
        self.dropout = dropout

    def forward(self, X, adj):
        X = F.relu(self.ncn1(X,adj))
        X = F.dropout(X, self.dropout, training=self.training)
        return F.log_softmax(self.ncn2(X, adj), dim=1)

In [0]:
adj, node_features, node_labels, idx_train, idx_val, idx_test = load_cora()


use_GPU = False
device = torch.device('cuda:0' if torch.cuda.is_available() and use_GPU else 'cpu')

adj = torch.from_numpy(np.copy(adj)).to(device).float()
node_features = torch.from_numpy(np.copy(node_features)).to(device).float()
# node_labels = torch.from_numpy(np.copy(node_labels)).to(device).float()
node_labels = torch.LongTensor(node_labels).to(device)

idx_train = torch.LongTensor(idx_train).to(device)
idx_val = torch.LongTensor(idx_val).to(device)
idx_test = torch.LongTensor(idx_test).to(device)

In [0]:
model = NCN(in_size=node_features.shape[1], n_hid=128, out_size=7, dropout = 0.5,
            edges_size = adj.shape[-1], bias = True, device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [0]:
import time

for epoch in range(100):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(node_features, adj)
    loss_train = F.nll_loss(output[idx_train], node_labels[idx_train])
    acc_train = accuracy(output[idx_train], node_labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # Evaluate validation set performance separately,
    # deactivates dropout during validation run.
    model.eval()
    output = model(node_features, adj)

    loss_val = F.nll_loss(output[idx_val], node_labels[idx_val])
    acc_val = accuracy(output[idx_val], node_labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.6f}'.format(loss_train.item()),
          'acc_train: {:.6f}'.format(acc_train.item()),
          'loss_val: {:.6f}'.format(loss_val.item()),
          'acc_val: {:.6f}'.format(acc_val.item()),
          'time: {:.6f}s'.format(time.time() - t))

Epoch: 0001 loss_train: 1.946966 acc_train: 0.135714 loss_val: 2.117717 acc_val: 0.376667 time: 1.980345s
Epoch: 0002 loss_train: 1.773531 acc_train: 0.357143 loss_val: 3.091569 acc_val: 0.293333 time: 1.770749s
Epoch: 0003 loss_train: 2.546145 acc_train: 0.464286 loss_val: 2.014833 acc_val: 0.503333 time: 1.743530s
Epoch: 0004 loss_train: 1.127110 acc_train: 0.685714 loss_val: 1.377938 acc_val: 0.663333 time: 1.726611s
Epoch: 0005 loss_train: 0.751519 acc_train: 0.800000 loss_val: 1.215742 acc_val: 0.726667 time: 1.723158s
Epoch: 0006 loss_train: 0.593705 acc_train: 0.892857 loss_val: 1.252734 acc_val: 0.703333 time: 1.726669s
Epoch: 0007 loss_train: 0.555923 acc_train: 0.892857 loss_val: 1.265209 acc_val: 0.710000 time: 1.709360s


KeyboardInterrupt: ignored

In [0]:
node_labels[idx_train]

tensor([[0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0.,

In [0]:
#a=adj f=features
a=np.stack([[[1,0,1],[1,0,0],[-1,-1,-1]], [[0,-1,0],[0,0,1],[0,1,0]]], axis=2)
# a = np.array([[[1,0],[0,-1],[0,0]]]) #2 tipos de arestas, 1 no
f=np.array([[1,2,3,4,5],[10,20,30,40,50],[5,10,15,20,25]]) #5 features por no, 3 nos

n_out=3
w = np.ones((n_out,a.shape[-1]*f.shape[-1]))
w[0,-n_out:]=0
w[1,:-n_out]=0
# w =np.eye(f.shape[-1])
# w = np.array([[[1,1,1,1,1],[0,0,0,0,0]],[[0,0,0,0,0],[1,1,1,1,1]],[[1,0,1,0,1],[0,1,0,1,0]]])
# tem que transformar de 5 canais para n

print(f"a: {a}")
print(a.shape)
print(f"f: {f}")
print(f"w: {w}")
# print(w.shape)
# print("\n")

# tmp=f.T@a
tmp = f.T@a
print(tmp)
# print(tmp.shape)

tmp = tmp.reshape(tmp.shape[0],-1)
print(f"tmp: {tmp}")
print(f"tmp reshaped: {tmp.shape}")
# print(f"w: {w.shape}")
res = tmp@w.T
print(f"res: {res}")

a: [[[ 1  0]
  [ 0 -1]
  [ 1  0]]

 [[ 1  0]
  [ 0  0]
  [ 0  1]]

 [[-1  0]
  [-1  1]
  [-1  0]]]
(3, 3, 2)
f: [[ 1  2  3  4  5]
 [10 20 30 40 50]
 [ 5 10 15 20 25]]
w: [[1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[[  6 -10]
  [ 12 -20]
  [ 18 -30]
  [ 24 -40]
  [ 30 -50]]

 [[  1   5]
  [  2  10]
  [  3  15]
  [  4  20]
  [  5  25]]

 [[-16  10]
  [-32  20]
  [-48  30]
  [-64  40]
  [-80  50]]]
tmp: [[  6 -10  12 -20  18 -30  24 -40  30 -50]
 [  1   5   2  10   3  15   4  20   5  25]
 [-16  10 -32  20 -48  30 -64  40 -80  50]]
tmp reshaped: (3, 10)
res: [[   0.  -60.  -60.]
 [  40.   50.   90.]
 [-100.   10.  -90.]]


In [0]:
device='cpu'
adj = torch.from_numpy(np.copy(a)).to(device).float()
feat = torch.from_numpy(np.copy(f)).to(device).float()
weights = torch.from_numpy(np.copy(w)).to(device).float()

# adj = adj.to_sparse()

print(adj)
print(feat)
print(weights,'\n\n')

tmp = feat.T@adj
print(f"tmp_shape before view: {tmp.shape}")
print(tmp)
tmp = tmp.view(tmp.shape[0], -1)
print(tmp,'\n\n')
out = torch.matmul(tmp,weights.T)
print(out)

NameError: ignored

In [0]:
device='cpu'
# a=np.stack([[[1,0,1],[1,0,0],[-1,-1,-1]], [[0,-1,0],[0,0,1],[0,1,0]]], axis=0)
a=np.stack([[[1,0,1],[1,0,0]], [[0,-1,0],[0,0,1]]], axis=0)
n_out=4
w = np.ones((n_out,a.shape[0]*f.shape[-1]))
w[0,-n_out:]=0
w[1,:-n_out]=0

adj = torch.from_numpy(np.copy(a)).to(device).float()
feat = torch.from_numpy(np.copy(f)).to(device).float()
weights = torch.from_numpy(np.copy(w)).to(device).float()

print(adj)
print(feat)
tmp=adj@feat
print(tmp)
tmp=tmp.permute(1,0,2).reshape(-1,2,5)
print(tmp)
print(weights)
# tmp@weights.T

tensor([[[ 1.,  0.,  1.],
         [ 1.,  0.,  0.]],

        [[ 0., -1.,  0.],
         [ 0.,  0.,  1.]]])
tensor([[ 1.,  2.,  3.,  4.,  5.],
        [10., 20., 30., 40., 50.],
        [ 5., 10., 15., 20., 25.]])
tensor([[[  6.,  12.,  18.,  24.,  30.],
         [  1.,   2.,   3.,   4.,   5.]],

        [[-10., -20., -30., -40., -50.],
         [  5.,  10.,  15.,  20.,  25.]]])
tensor([[[  6.,  12.,  18.,  24.,  30.],
         [-10., -20., -30., -40., -50.]],

        [[  1.,   2.,   3.,   4.,   5.],
         [  5.,  10.,  15.,  20.,  25.]]])
tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [0]:
device='cpu'
a=np.stack([[[1,0,1],[1,0,0],[-1,-1,-1]], [[0,-1,0],[0,0,1],[0,1,0]]], axis=0)
# a=np.stack([[[1,0,1],[1,0,0]], [[0,-1,0],[0,0,1]]], axis=0)
n_out=3
w = np.ones((n_out,a.shape[0]*f.shape[-1]))
w[0,-n_out:]=0
w[1,:-n_out]=0

adj = torch.from_numpy(np.copy(a)).to(device).float()
feat = torch.from_numpy(np.copy(f)).to(device).float()
weights = torch.from_numpy(np.copy(w)).to(device).float()

print(adj)
print(feat)
tmp=adj@feat
print(tmp)
# print(tmp.)
# a@f
print(weights)

tensor([[[ 1.,  0.,  1.],
         [ 1.,  0.,  0.],
         [-1., -1., -1.]],

        [[ 0., -1.,  0.],
         [ 0.,  0.,  1.],
         [ 0.,  1.,  0.]]])
tensor([[ 1.,  2.,  3.,  4.,  5.],
        [10., 20., 30., 40., 50.],
        [ 5., 10., 15., 20., 25.]])
tensor([[[  6.,  12.,  18.,  24.,  30.],
         [  1.,   2.,   3.,   4.,   5.],
         [-16., -32., -48., -64., -80.]],

        [[-10., -20., -30., -40., -50.],
         [  5.,  10.,  15.,  20.,  25.],
         [ 10.,  20.,  30.,  40.,  50.]]])
tensor([[1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [0]:
device='cpu'
adj = torch.from_numpy(np.copy(a)).to(device).float()
feat = torch.from_numpy(np.copy(f)).to(device).float()
weights = torch.from_numpy(np.copy(w)).to(device).float()

print(f"adj_shape:{adj.shape}\nadj_t_shape:{adj.T.shape}")
print(f"feat_shape:{feat.shape}\nfeat_t_shape:{feat.T.shape}")
adj = adj.T.to_sparse()
print(adj[0])
# print(adj[0]@feat)
# print(f.T@a)

# adj_s = adj.to_sparse()
# feat_s = feat.to_sparse()

# adjt_s = adj.T.to_sparse()
# featt_s = feat.T.to_sparse()

# torch.sparse.mm(adj_s, feat.T)

adj_shape:torch.Size([2, 3, 2])
adj_t_shape:torch.Size([2, 3, 2])
feat_shape:torch.Size([3, 5])
feat_t_shape:torch.Size([5, 3])
tensor(indices=tensor([[0, 0, 2],
                       [0, 1, 0]]),
       values=tensor([1., 1., 1.]),
       size=(3, 2), nnz=3, layout=torch.sparse_coo)


In [0]:
adj.shape

In [0]:
i = torch.LongTensor([[0, 1, 1],
                          [2, 0, 2]])
v = torch.FloatTensor([3, 4, 5])
torch.sparse.FloatTensor(i, v, torch.Size([2,3])).to_dense()

tensor([[0., 0., 3.],
        [4., 0., 5.]])

In [0]:
t1 = torch.randn((3,4,5)).to_sparse()

In [0]:
t1

tensor(indices=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2],
                       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3,
                        3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3,
                        3, 3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3,
                        3, 3, 3],
                       [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
                        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
                        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
                        2, 3, 4]]),
       values=tensor([-0.7561, -1.0995,  0.7820, -2.8600,  1.5064,  2.0398,
                      -0.2025,  1.4670, -0.8606,  0.3663, -0.8735,  0.0400,
               